# **DEPENDENCIES**

In [ ]:
# Local import
# import sys
# sys.path.append('..')

# Import on colab
!pip install git+https://github.com/adityaprakash-work/DreamWalker.git

from google.colab import drive
drive.mount('/content/drive')

In [2]:
import torch
import dreamwalker as dw

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# **DATASET**

In [5]:
# Run this cell to load from an online source
dataset_url = "https://www.kaggle.com/datasets/ifigotin/imagenetmini-1000/data"
dataset_dir = "/content/dataset"
dw.utils.datasets.download(dataset_url, dataset_dir)

100%|██████████| 3.92G/3.92G [00:44<00:00, 95.3MB/s]


In [16]:
train_dir = "/content/dataset/imagenetmini-1000/imagenet-mini/train"
valid_dir = "/content/dataset/imagenetmini-1000/imagenet-mini/val"
if valid_dir is None:
    dataset = dw.utils.datasets.ImageStream(train_dir, ext="JPEG")
    train_loader, valid_loader = dw.utils.datasets.get_loaders(
        dataset, return_valid=True, valid_size=0.2
    )

else:
    train_dataset = dw.utils.datasets.ImageStream(train_dir, ext="JPEG")
    valid_dataset = dw.utils.datasets.ImageStream(valid_dir, ext="JPEG")
    train_loader = dw.utils.datasets.get_loaders(train_dataset, batch_size=16)
    valid_loader = dw.utils.datasets.get_loaders(valid_dataset, batch_size=16)

In [17]:
for data, _ in train_loader:
    print(data.shape)
    break

torch.Size([16, 3, 128, 128])


# **TRAINING**

In [18]:
load_path = None
save_path = "/content/checkpoints/vqvae0.pth"

if load_path is None:
    model = dw.models.vqvae.ResVQVAE()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    trainer = dw.models.vqvae.VQVAETrainer(
        model,
        optimizer,
        train_loader,
        valid_loader,
        device=DEVICE,
        load_checkpoint_path=None,
    )
else:
    trainer = dw.models.vqvae.VQVAETrainer(
        train_loader=train_loader,
        valid_loader=valid_loader,
        device=DEVICE,
        load_checkpoint_path=load_path,
    )

In [ ]:
epochs = 100
monitors = [dw.utils.training.BestModelCheckpoint(save_path=save_path)]

trainer.train(epochs, monitors=monitors)